In [1]:
import os

In [2]:
%pwd

'e:\\NutBodyslam053\\work\\Projects\\MLOps\\BD528-software_engineering\\research'

In [3]:
os.chdir("../")
%pwd

'e:\\NutBodyslam053\\work\\Projects\\MLOps\\BD528-software_engineering'

In [4]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/NutBodyslam053/BD528-software_engineering.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "NutBodyslam053"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "b85bafd69d98861fee89f5bf70dc5f62cf41c2e5"

print("MLFLOW_TRACKING_URI:", os.environ.get("MLFLOW_TRACKING_URI"))
print("MLFLOW_TRACKING_USERNAME:", os.environ.get("MLFLOW_TRACKING_USERNAME"))
print("MLFLOW_TRACKING_PASSWORD:", os.environ.get("MLFLOW_TRACKING_PASSWORD"))

MLFLOW_TRACKING_URI: https://dagshub.com/NutBodyslam053/BD528-software_engineering.mlflow
MLFLOW_TRACKING_USERNAME: NutBodyslam053
MLFLOW_TRACKING_PASSWORD: b85bafd69d98861fee89f5bf70dc5f62cf41c2e5


## entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

## config

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH) -> None:

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/NutBodyslam053/BD528-software_engineering.mlflow"
        )

        return model_evaluation_config

## components

In [7]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mlProject.utils.common import save_json
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib
from pathlib import Path


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        return accuracy, precision, recall, f1

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (accuracy, precision, recall, f1) = self.eval_metrics(
                test_y, predicted_qualities)

            scores = {"accuracy": accuracy,
                      "precision": precision, "recall": recall, "f1": f1}
            
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1", f1)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(
                    model, "model", registered_model_name="RandomForestClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")

## pipeline

In [8]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="setuptools")


try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

2023-10-31 14:58:00,350: INFO: common: yaml file: config\config.yaml loaded successfully
2023-10-31 14:58:00,354: INFO: common: yaml file: params.yaml loaded successfully
2023-10-31 14:58:00,359: INFO: common: yaml file: schema.yaml loaded successfully
2023-10-31 14:58:00,360: INFO: common: created directory at: artifacts
2023-10-31 14:58:00,362: INFO: common: created directory at: artifacts/05_model_evaluation
2023-10-31 14:58:02,857: INFO: common: json file saved at: artifacts\05_model_evaluation\metrics.json


e:\NutBodyslam053\work\Projects\MLOps\BD528-software_engineering\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2023/10/31 14:58:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestClassifier, version 4
Created version '4' of model 'RandomForestClassifier'.
